In [1]:
from sql import Sql
import pandas
from pandas.io.json import json_normalize
import pyodbc
import json

In [14]:
def get_db_con():
    # make a connection string
    return pyodbc.connect('Trusted_Connection=yes',
                             driver='{ODBC Driver 13 for SQL Server}',
                             server='localhost',
                             database='Weather')

In [2]:
# open the weather json file
try:
    fileName = 'vancouverweather.json' #'C:\\Users\\jmaitland\\Source\\Repos\\VesselPrediction\\DataParsing\\vancouverweather.json'
    file = open(fileName)
    sfile = file.read()
except FileNotFoundError:
    print("The file {} could not be found".format(fileName))

In [3]:
# load json file into a list
raw_json = json.loads(sfile)

In [4]:
result = json_normalize(raw_json, errors='ignore')

In [5]:
weather_df = result[['dt', 'weather']]

In [6]:
index = list(range(46290))
columns = ['dt', 'descr', 'main', 'icon', 'id']

In [7]:
weather_df2 = pandas.DataFrame(index=index, columns=columns)

In [8]:
i = 0
j = 0
this_row = 0
for index, row in weather_df.iterrows():
    for item in row['weather']:
        weather_df2['dt'][i] = row['dt']
        weather_df2['descr'][i] = item['description']
        weather_df2['main'][i] = item['main']
        weather_df2['icon'][i] = item['icon']
        weather_df2['id'][i] = item['id']
        i += 1
    j += 1

In [20]:
#declare columns for sql insert
con = get_db_con()
cursor = con.cursor()
sql = Sql()
columns = ['LinuxTime', 'Descr', 'Main', 'Icon', 'WeatherID']
query = sql.get_insert(table_name='weather.weather', columns=columns)

In [22]:
# loop through the df and insert to sql server
for index, row in weather_df2.iterrows():
    values = [int(row['dt']), str(row['descr']), str(row['main']), str(row['icon']),int(row['id'])]
    cursor.execute(query, values)
cursor.commit()

In [37]:
columns = ['cityid', 'clouds', 'LinuxTime', 'dtos', 'humidity', 'pressure', 'temp', 
           'MaxTemp', 'MinTemp', 'rain1h', 'rain24h', 'rain3h', 'rainToday', 
           'snow1h','snow3h', 'winddeg', 'windspeed']
query = sql.get_insert(table_name='weather.raw', columns=columns)

In [47]:
for i, r in result.iterrows():
    if type(r['rain.3h']) != float:
        print(i)

In [49]:
for index, row in result.iterrows():
    values = [int(row['city_id']), 
              int(row['clouds.all']),
              int(row['dt']),
              str(row['dt_iso']),
              int(row['main.humidity']),
              int(row['main.pressure']),
              str(row['main.temp']),
              str(row['main.temp_max']),
              str(row['main.temp_min']),
              str(row['rain.1h']),
              str(row['rain.24h']),
              str(row['rain.3h']),
              str(row['rain.today']),
              str(row['snow.1h']),
              str(row['snow.3h']),
              int(row['wind.deg']),
              int(row['wind.speed'])]
    cursor.execute(query, values)
#cursor.commit()

In [50]:
cursor.commit()